In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_classif, SelectKBest, RFE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('Dataset_clustering.csv')
df.head()

,Ship_Type,Route_Type,Engine_Type,Maintenance_Status,Weather_Condition,Speed_Over_Ground_knots,Engine_Power_kW,Distance_Traveled_nm,Cargo_Weight_tons,Operational_Cost_USD,Revenue_per_Voyage_USD,Turnaround_Time_hours,Efficiency_nm_per_kWh,Seasonal_Impact_Score,Weekly_Voyage_Count,Average_Load_Percentage,Cluster
0,Container Ship,Long-haul,Heavy Fuel Oil (HFO),Critical,Moderate,12.597558,2062.983982,1030.943616,1959.017882,483832.354540,292183.273104,25.867077,1.455179,1.415653,1.0,93.769249,3
1,Fish Carrier,Short-haul,Steam Turbine,Good,Rough,10.387580,1796.057415,1060.486382,162.394712,483388.000509,883765.787360,63.248196,0.290361,0.885648,6.0,93.895372,2
2,Container Ship,Long-haul,Diesel,Fair,Moderate,20.749747,1648.556685,658.874144,178.040917,448543.404044,394018.746904,49.418150,0.499595,1.405813,9.0,96.218244,3
3,Bulk Carrier,Transoceanic,Steam Turbine,Fair,Moderate,21.055102,915.261795,1126.822519,1737.385346,261349.605449,87551.375175,22.409110,0.702906,1.370704,1.0,66.193698,0
4,Fish Carrier,Transoceanic,Diesel,Fair,Moderate,13.742777,1089.721803,1445.281159,260.595103,287718.375160,676121.459632,64.158231,1.331343,0.583383,8.0,80.008581,2


In [3]:
object_df = df.select_dtypes(include='object').columns.tolist()
numeric_df = df.select_dtypes(exclude='object').columns.tolist()[:-1]

In [4]:
data_encoded = df.copy()
for col in object_df:
    le = LabelEncoder()
    data_encoded[col] = le.fit_transform(data_encoded[col])
    
data_encoded.head()

,Ship_Type,Route_Type,Engine_Type,Maintenance_Status,Weather_Condition,Speed_Over_Ground_knots,Engine_Power_kW,Distance_Traveled_nm,Cargo_Weight_tons,Operational_Cost_USD,Revenue_per_Voyage_USD,Turnaround_Time_hours,Efficiency_nm_per_kWh,Seasonal_Impact_Score,Weekly_Voyage_Count,Average_Load_Percentage,Cluster
0,1,1,1,0,1,12.597558,2062.983982,1030.943616,1959.017882,483832.354540,292183.273104,25.867077,1.455179,1.415653,1.0,93.769249,3
1,2,2,2,2,2,10.387580,1796.057415,1060.486382,162.394712,483388.000509,883765.787360,63.248196,0.290361,0.885648,6.0,93.895372,2
2,1,1,0,1,1,20.749747,1648.556685,658.874144,178.040917,448543.404044,394018.746904,49.418150,0.499595,1.405813,9.0,96.218244,3
3,0,3,2,1,1,21.055102,915.261795,1126.822519,1737.385346,261349.605449,87551.375175,22.409110,0.702906,1.370704,1.0,66.193698,0
4,2,3,0,1,1,13.742777,1089.721803,1445.281159,260.595103,287718.375160,676121.459632,64.158231,1.331343,0.583383,8.0,80.008581,2


In [5]:
def feature_selection_mi(df, target):
    # Pilih fitur-fitur terbaik berdasarkan mutual information
    mi_scores = mutual_info_classif(df.drop(target, axis=1), df[target])
    mi_df = pd.DataFrame({'fitur': df.drop(target, axis=1).columns, 'mi_score': mi_scores})
    mi_df.sort_values(by='mi_score', ascending=False, inplace=True)
    
    # Pilih top K fitur
    k = 5  # Sesuaikan nilai K
    top_features = mi_df['fitur'].head(k).tolist()
    
    return top_features

In [6]:
top_features = feature_selection_mi(data_encoded, 'Cluster')

print("Fitur-fitur terbaik:", top_features)

Fitur-fitur terbaik: ['Ship_Type', 'Route_Type', 'Engine_Type', 'Efficiency_nm_per_kWh', 'Weekly_Voyage_Count']
